https://arxiv.org/pdf/2110.01368.pdf

https://arxiv.org/pdf/1911.03380.pdf

https://deliverypdf.ssrn.com/delivery.php?ID=773124118097097119026017080102079065057081049043000029071005117069008065110098087085056017039010001111005112084088095103006007043087058092007068069090091015080092022071021048008071122094090110120127079071113126114121118104123004007070096119111104095125&EXT=pdf&INDEX=TRUE

Let $\Sigma_t$ denote portfolio of the instruments under our managment
$$\Sigma_t = n_i(t) \cdot I_i(t),$$
where $n_i(t)$ -- position in the instrument $i$, $I_i(t)$ -- its present value

$$\dot\Sigma(t) = \dot n_i I_i + n_i \dot I_i$$

So we must implement Instruments' dynamics and portolio strategy to control $n_i$ 

In [1]:
import numpy as np
import pandas as pd
from LPtools.utils import Adapter
from LPtools.clients import Client
from LPtools.instruments import UniPool, Bond, Portfolio, Position, Cash, Holder

In [2]:
zcurve = pd.read_csv('zcyc_rates.csv', index_col=0, parse_dates=True, infer_datetime_format=True, na_filter=False)
#zcurve = zcurve.reindex(pd.date_range(zcurve.index.min(), zcurve.index.max()), method='pad').to_csv('zcyc_rates.csv')

adapter = Adapter(pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
                  symbol="ETHUSDT",
                  zcurve=zcurve)

response = adapter(100)
#response.token1Price = 1. # for some reason retrieved USDC value is equal to ~0.0003,  $1 expected

In [3]:
# however for some reason retrieved USDC value is equal to ~0.0003 instead of $1 expected
# it leads to wrong numbers from graphql for the other columns, binance data is ok.

# it is also interesting how uniswap price differs from binance market price

# or I don't understand sqrtPrice and other fields values correctly

response['difference'] = 1 / response.token1Price
response.relative_price = response.relative_price / response.difference
response.liquidity =  response.liquidity / response.difference
response.token1Price = 1

#after correction there are still errors left, so quering graphql and
#         making sense out of units of measure is still required

In [4]:
runner = response.itertuples()
print(response.shape)
response.head()

(100, 15)


,fundingRate,Open,Volume,date,feesUSD,liquidity,sqrtPrice,token0Price,token1Price,tvlUSD,volumeUSD,relative_price,mark,risk_free_rate,difference
2021-08-23,0.001587,3241.29,2430852.991,1.629677e+09,162011.187199,3.975834e+15,1.374390e+33,3323.068541,1,1.472374e+08,3.240224e+08,3323.068541,3241.19000000,6.42,3323.068541
2021-08-24,0.001189,3323.00,2520587.128,1.629763e+09,196604.907005,4.016806e+15,1.406044e+33,3175.128922,1,1.457254e+08,3.932098e+08,3175.128922,3323.01000000,6.43,3175.128922
2021-08-25,0.000465,3173.99,2405504.816,1.629850e+09,194457.274305,5.631325e+15,1.394881e+33,3226.152108,1,1.522418e+08,3.889145e+08,3226.152108,3173.98000000,6.39,3226.152108
2021-08-26,0.000300,3227.90,2360877.641,1.629936e+09,191172.517182,4.422989e+15,1.423204e+33,3099.022574,1,1.449578e+08,3.823450e+08,3099.022574,3228.21000000,6.45,3099.022574
2021-08-27,0.000300,3093.81,2227370.879,1.630022e+09,172471.174069,7.443776e+15,1.385700e+33,3269.043498,1,1.518801e+08,3.449423e+08,3269.043498,3092.99393436,6.44,3269.043498


In [5]:
init_state = next(runner)

In [6]:
cash = Cash(0)

pool = Position(tag='pool', instrument=UniPool(init_state, 2000, 5000, .005))
bond = Position(tag='bond', instrument=Bond(init_state))

cash += pool.rebalance(1000) + bond.rebalance(-1000) # create positions, suffer transaction costs

positions = [pool, bond]
balancer = Holder(positions)
portfolio = Portfolio(balancer, cash=cash, positions=positions)

In [7]:
portfolio.rollout(runner)

In [8]:
portfolio.summary

pool_last_val = 202.57963261481555
bond_last_val = -1019.8307372217124
total_value = -837.2511045928053


,pool,bond,payments,transaction_costs
0,0.100879,1.000178,8.691412e-11,0
1,0.104789,1.000357,6.130693e-11,0
2,0.094988,1.000535,7.673681e-11,0
3,0.108052,1.000714,4.112169e-11,0
4,0.106198,1.000893,2.713775e-11,0
...,...,...,...,...
94,0.163532,1.018701,2.293112e-10,0
95,0.167321,1.018930,1.015536e-10,0
96,0.180534,1.019158,1.140112e-10,0
97,0.190465,1.019387,9.035786e-11,0
